## Coursework 2
Objective:
(1) obtain practical knowledge and hands-on understanding of the basic concepts in Generative Adversarial Nets(GAN);
(2) obtain practical experience on how to implement basic GAN using tensorflow.

Tasks: three subtasks are involved:
1. Coding: to add your code blocks in the required sections; (30% of this CW)
2. Demonstrating: to answer TWO questions randomly selected from below
during the lab demo session in WK11; (20% of this CW)
3. Report: to complete the questions in report. (50% of this CW)

Basic material:
Some of online materials for tensorflow-code may help you better complete this coursework (if you not familiar with tensorflow, you can follow this step by step) https://www.tensorflow.org/versions/r1.1/get_started/mnist/beginners https://github.com/floydhub/tensorflow-notebooks- examples/blob/master/3_NeuralNetworks/autoencoder.ipynb

## The questions to think over:

**1. What are two basic part in generative model?**<br>
* a generative model G
* a discriminative model D 

**2. What is the specific objective of these two part?**<br>
* generative model G - captures the data distribution
* discriminative model D - estimates the probability that a sample came from the training data rather than G

**3. What is the basic loss function of GAN?**<br>
Loss function or cost function is a function that maps an event or values of one or more variables onto a real number intuitively representing some "cost" associated with the event. Need to minimize a loss function.

The discriminator(G_sample) wants the predictions on the “real” data to be all ones and the predictions on the “fake” data from the generator to be all zeros. The generator wants the discriminator’s predictions to be all ones, as per definition, we want to maximize the probability of real data.

Maximizing tf.reduce_mean(tf.log(D_fake)).

**4. What is training process of basic GAN model?**<br>
Training the GAN means Training the Generator. The generator initially produces garbage images, and the loss value is high. So, the back-propagation updates the generator’s weights to produce more realistic images as the training continues. This is how the generator is trained via training the GAN.

In [19]:
#!pip install dask --upgrade
import os, time, itertools, imageio, pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
# G(z)
def generator(x):
    # initializers
    w_init = tf.truncated_normal_initializer(mean=0, stddev=0.02)
    b_init = tf.constant_initializer(0.)
    # 1st hidden layer
    w0 = tf.get_variable('G_w0', [x.get_shape()[1], 128], initializer=w_init)
    b0 = tf.get_variable('G_b0', [128], initializer=b_init)
    h0 = tf.nn.relu(tf.matmul(x, w0) + b0)
    # output hidden layer
    w1 = tf.get_variable('G_w1', [h0.get_shape()[1], 784], initializer=w_init)
    b1 = tf.get_variable('G_b1', [784], initializer=b_init)
    o = tf.nn.tanh(tf.matmul(h0, w1) + b1)
    return o
    ### Code:ToDO( Change the architecture as CW2 Guidance required)

In [5]:
# D(x)
def discriminator(x, drop_out):
    # initializers
    w_init = tf.truncated_normal_initializer(mean=0, stddev=0.02)
    b_init = tf.constant_initializer(0.)
    # 1st hidden layer
    w0 = tf.get_variable('D_w0', [x.get_shape()[1], 784], initializer=w_init)
    b0 = tf.get_variable('D_b0', [784], initializer=b_init)
    h0 = tf.nn.relu(tf.matmul(x, w0) + b0)
    # output layer
    w1 = tf.get_variable('D_w1', [h0.get_shape()[1], 1], initializer=w_init)
    b1 = tf.get_variable('D_b1', [1], initializer=b_init)
    o = tf.sigmoid(tf.matmul(h0, w1) + b1)
    ###  Code: ToDO( Change the architecture as CW2 Guidance required)

    return o

In [6]:
def show_result(num_epoch, show = False, save = False, path = 'result.png'):
    z_ = np.random.normal(0, 1, (25, 100))    # z_ is the input of generator, every epochs will random produce input
    ##Code:ToDo complete the rest of part

def show_train_hist(hist, show = False, save = False, path = 'Train_hist.png'):
    x = range(len(hist['D_losses']))

    y1 = hist['D_losses']
    y2 = hist['G_losses']

    plt.plot(x, y1, label='D_loss')
    plt.plot(x, y2, label='G_loss')

    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

In [7]:
# training parameters
batch_size = 100
lr = 0.0002
train_epoch = 100

# load MNIST
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
train_set = (mnist.train.images - 0.5) / 0.5  # normalization; range: -1 ~ 1

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [8]:
# networks : generator
with tf.variable_scope('G'):
    z = tf.placeholder(tf.float32, shape=(None, 100))
    G_z = generator(z)
# networks : discriminator
with tf.variable_scope('D') as scope:
    drop_out = tf.placeholder(dtype=tf.float32, name='drop_out')
    x = tf.placeholder(tf.float32, shape=(None, 784))
    D_real = discriminator(x, drop_out)
    scope.reuse_variables()
    D_fake = discriminator(G_z, drop_out)

In [9]:
# loss for each network
eps = 1e-2
D_loss = tf.reduce_mean(-tf.log(D_real + eps) - tf.log(1 - D_fake + eps))
G_loss = tf.reduce_mean(-tf.log(D_fake + eps))

# trainable variables for each network
t_vars = tf.trainable_variables()
D_vars = [var for var in t_vars if 'D_' in var.name]
G_vars = [var for var in t_vars if 'G_' in var.name]

In [10]:
# optimizer for each network
D_optim = tf.train.AdamOptimizer(lr).minimize(D_loss, var_list=D_vars)
G_optim = tf.train.AdamOptimizer(lr).minimize(G_loss, var_list=G_vars)

# open session and initialize all variables
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [11]:
# results save folder
if not os.path.isdir('MNIST_GAN_results'):
    os.mkdir('MNIST_GAN_results')
if not os.path.isdir('MNIST_GAN_results/results'):
    os.mkdir('MNIST_GAN_results/results')
train_hist = {}
train_hist['D_losses'] = []
train_hist['G_losses'] = []
train_hist['per_epoch_ptimes'] = []
train_hist['total_ptime'] = []

In [12]:
# training-loop
np.random.seed(int(time.time()))
start_time = time.time()
for epoch in range(train_epoch):
    G_losses = []
    D_losses = []
    epoch_start_time = time.time()
    for iter in range(train_set.shape[0] // batch_size):
        # update discriminator
        x_ = train_set[iter*batch_size:(iter+1)*batch_size]
        z_ = np.random.normal(0, 1, (batch_size, 100))

        loss_d_, _ = sess.run([D_loss, D_optim], {x: x_, z: z_, drop_out: 0.3})
        D_losses.append(loss_d_)

        # update generator
        z_ = np.random.normal(0, 1, (batch_size, 100))
        loss_g_, _ = sess.run([G_loss, G_optim], {z: z_, drop_out: 0.3})
        G_losses.append(loss_g_)

    epoch_end_time = time.time()
    per_epoch_ptime = epoch_end_time - epoch_start_time
    print('[%d/%d] - ptime: %.2f loss_d: %.3f, loss_g: %.3f' % ((epoch + 1), train_epoch, per_epoch_ptime, np.mean(D_losses), np.mean(G_losses)))

    ### Code: TODO Code complet show_result function)
    ###p = 'MNIST_GAN_results/results/MNIST_GAN_' + str(epoch + 1) + '.png'
    ###show_result((epoch + 1), save=True, path=p)
    train_hist['D_losses'].append(np.mean(D_losses))
    train_hist['G_losses'].append(np.mean(G_losses))
    train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
end_time = time.time()
total_ptime = end_time - start_time
train_hist['total_ptime'].append(total_ptime)
print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(train_hist['per_epoch_ptimes']), train_epoch, total_ptime))
print("Training finish!... save training results")
with open('MNIST_GAN_results/train_hist.pkl', 'wb') as f:
    pickle.dump(train_hist, f)
show_train_hist(train_hist, save=True, path='MNIST_GAN_results/MNIST_GAN_train_hist.png')
images = []
sess.close()

[1/100] - ptime: 27.36 loss_d: 0.488, loss_g: 1.061
[2/100] - ptime: 24.40 loss_d: 0.182, loss_g: 2.142
[3/100] - ptime: 23.63 loss_d: 0.292, loss_g: 2.458
[4/100] - ptime: 26.08 loss_d: 0.474, loss_g: 2.243
[5/100] - ptime: 23.52 loss_d: 0.292, loss_g: 2.518
[6/100] - ptime: 24.28 loss_d: 0.366, loss_g: 2.410
[7/100] - ptime: 26.22 loss_d: 0.398, loss_g: 2.278
[8/100] - ptime: 26.58 loss_d: 0.560, loss_g: 2.186
[9/100] - ptime: 24.71 loss_d: 0.715, loss_g: 1.925
[10/100] - ptime: 26.58 loss_d: 0.740, loss_g: 1.823
[11/100] - ptime: 25.05 loss_d: 0.767, loss_g: 1.623
[12/100] - ptime: 27.54 loss_d: 0.735, loss_g: 1.754
[13/100] - ptime: 25.09 loss_d: 0.672, loss_g: 1.928
[14/100] - ptime: 28.52 loss_d: 0.589, loss_g: 2.055
[15/100] - ptime: 24.77 loss_d: 0.785, loss_g: 1.666
[16/100] - ptime: 22.83 loss_d: 0.839, loss_g: 1.740
[17/100] - ptime: 22.00 loss_d: 0.893, loss_g: 1.567
[18/100] - ptime: 21.53 loss_d: 0.864, loss_g: 1.563
[19/100] - ptime: 21.60 loss_d: 0.844, loss_g: 1.568
[2

## The excises to conduct:

**1. Understanding the  framework.**<br>
1) Dataset: What is dataset being used? How can you load this dataset?
#MNIST dataset was used
from tensorflow.examples.tutorials.mnist import input_data
2) Model: Can you plot/draw the basic architecture in this case?
Bellow
3) Loss: What is loss function in this example?
eps = 1e-2
D_loss = tf.reduce_mean(-tf.log(D_real + eps) - tf.log(1 - D_fake + eps))
G_loss = tf.reduce_mean(-tf.log(D_fake + eps))

4) Training: How is this network being trained?

5) Test: How do you test this model?
G learns to make data that is indistiguishable from real data to the discriminator.

**2. Change the learning rate to 0.01 and train for a few epochs to understand
how learning rate will influence the model outcome.**<br>
Initial learning rate was lr = 0.0002 :
[100/100] - ptime: 22.50 loss_d: 0.575, loss_g: 3.076
Avg per epoch ptime: 4.09, total 100 epochs ptime: 409.31


**3. Change the batch size to 256, and discuss how the batch size influence the
model performance regarding its training speed and test accuracy.**<br>
Loss function or cost function is a function that maps an event or values of one or more variables onto a real number intuitively representing some "cost" associated with the event. Need to minimize a loss function.

**4. Change the training epoch to 200, run it.**<br>
Training the GAN me

In [ ]:
#Model: plot/draw the basic architecture
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('arch.png')
imgplot = plt.imshow(img)
plt.show()